In [ ]:
#1

In [ ]:
#CSV読み込み
import os
import pandas as pd
from thefuzz import process  # fuzzy string matching

# CSVが入っているディレクトリ
directory = "your_csv_directory"
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

# CSVのすべての列名を集める
all_columns = set()
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path, nrows=0)  # データは読まずに列だけ取得
    all_columns.update(df.columns.tolist())

print("=== 集めた列名 ===")
print(all_columns)


＃fuzzで似たような列名を集める
rename_map = {}
for col in all_columns:
    best_match, score = process.extractOne(col, standard_columns)
    if score >= 70:  # 類似度が70%以上ならマッチ
        rename_map[col] = best_match
    else:
        rename_map[col] = col

print(rename_map)


In [ ]:

# 辞書をDataFrameに変換
df_map = pd.DataFrame(list(rename_map.items()), columns=["元の列名", "マッチした標準列名"])

# CSVに保存
df_map.to_csv("column_mapping.csv", index=False, encoding="utf-8-sig")

print("column_mapping.csv に保存しました！")

In [ ]:
# 理想の標準列名を定義（自分で決める）
standard_columns = ["Name", "Age", "Email"]
# 各CSVを読み込み、列名を統一・順序もそろえる
normalized_dfs = []
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path)
    df = df.rename(columns=lambda x: rename_map.get(x, x))
    df = df.reindex(columns=standard_columns)  # 順序を統一
    normalized_dfs.append(df)

# すべて結合
merged_df = pd.concat(normalized_dfs, ignore_index=True)

print("=== 統一後のデータ ===")
print(merged_df.head())


In [ ]:
import os
import pandas as pd
from collections import Counter

directory = "your_csv_directory"
csv_files = [f for f in os.listdir(directory) if f.endswith(".csv")]

all_columns = []
for file in csv_files:
    file_path = os.path.join(directory, file)
    df = pd.read_csv(file_path, nrows=0)  # 列名だけ取得
    all_columns.extend(df.columns.tolist())

# 頻度を数える
counter = Counter(all_columns)

# よく出てくる列名を候補にする
print("=== 標準列名候補 ===")
for col, freq in counter.most_common(20):
    print(f"{col} : {freq} 回")
